In [ ]:
# Import basic libraries
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Show available data
import os
print("Available files:")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Import data
file_data = "/kaggle/input/laminar-coflow-flame-isf-f3a-pulsating/F3A_A090_f020Hz.csv"
data = pd.read_csv(file_data) 
data.head()

In [ ]:
# Download PyTools4OpenSMOKEpp library from github
# It will be available in the /kaggle/working/PyTools4OpenSMOKEpp folder
!git clone https://github.com/acuoci/PyTools4OpenSMOKEpp.git

In [ ]:
# Import PyTools4OpenSMOKEpp classes
import re
import sys
sys.path.insert(0, '/kaggle/working/PyTools4OpenSMOKEpp/src')
from KineticMechanism import KineticMechanism
from OpenSMOKEppXMLFile import OpenSMOKEppXMLFile

In [ ]:
# Import kinetic mechanism in XML format
kin_xml_file = "/kaggle/input/laminar-coflow-flame-isf-f3a-pulsating/kinetics.xml"
kinetics = KineticMechanism(kin_xml_file)

In [ ]:
# Examples of functions to access details of species included in the kinetic mechanism

# List of species in the mechanism
kinetics.species

# Index of a species in the kinetic mechanism (from 0)
kinetics.species.index("C10H8")

# Molecular weights of species (in kg/kmol)
kinetics.mws

# List of elements
kinetics.elements

# Atomic composition of a species
kinetics.atomic[kinetics.species.index("C10H8")]

# List of PAH12 species (soot precursors with 1-2 aromatic rings)
kinetics.pah12_list

# List of PAH34 species (soot precursors with 3-4 aromatic rings)
kinetics.pah34_list

# List of PAHLP species (soot precursors with more than 4 aromatic rings, also called Large Precursors)
kinetics.pahlp_list

# List of soot spherical particles
kinetics.sp_list

# List of soot aggregates
kinetics.agg_list

In [ ]:
# Examples of manipulation of data


# Transform mass fractions (Y) of C10H8 into mole fractions (X): X = Y*MWmix/MW
Y = data["C10H8"]
MWmix = data["MW[kg/kmol]"]
X = Y*MWmix/kinetics.mws[kinetics.species.index("C10H8")]

# Mass fraction of BINs having section index equal to 2 (i.e. BIN2A, BIN2B, BIN2C, BIN2AJ, BIN2BJ, BIN2CJ)
list_bins = []
for name in kinetics.species:
        if (re.match("BIN2[A-Z]", name)):
            list_bins.append(name)
Y = data[list_bins].sum(axis=1)

# List species having molecular weight < 100 kg/kmol and sum them all
list_species = (np.array(kinetics.species)[kinetics.mws < 100.]).tolist()
Y = data[list_species].sum(axis=1)

# Identify observations where the soot volume fraction is at least 0.1 ppm (i.e. 1e-7) 
# and plot soot volume fraction vs mixture fraction
sel_data = data[data['fvSoot'] > 1e-7]
plt.scatter(sel_data["csi"], sel_data["fvSoot"], s=1)

In [ ]:
# Plot 2D colored map of selected fields at a given time

import matplotlib.tri as tri

def Map2D(x,y,z):
        
    # Create grid values first
    xi = np.linspace(0., x.max(), 100)
    yi = np.linspace(0., y.max(), 100)

    # Linearly interpolate the data (x, y) on a grid defined by (xi, yi).
    triang = tri.Triangulation(x, y)
    interpolator = tri.LinearTriInterpolator(triang, z)
    Xi, Yi = np.meshgrid(xi, yi)
    zi = interpolator(Xi, Yi)

    # Plot 2D colored map
    fig, ax = plt.subplots(nrows=1)
    ax.contour(xi, yi, zi, levels=14, linewidths=0.5, colors='k')
    cntr = ax.contourf(xi, yi, zi, levels=11, cmap="RdBu_r")
    fig.colorbar(cntr, ax=ax)
    ax.set(xlim=(0, x.max()/3), ylim=(0, y.max()))
    
    
# Select data at a given time (for example a t t=0)
sel_data = data[data['t[s]'] == 0.]
x = sel_data["x[m]"]
y = sel_data["z[m]"]

# Map of temperature
Map2D(x,y,sel_data["T[K]"])

# Map of hydrogen mass fraction
Map2D(x,y,sel_data["H2"])
